# LLM Pipeline - Готовый пайплайн для работы с большими языковыми моделями

Этот notebook содержит готовый пайплайн для:
- Загрузки и использования LLM моделей
- Fine-tuning с использованием LoRA/QLoRA
- Генерации текста и чат-ботов
- Квантизации моделей

## 1. Установка зависимостей

In [ ]:
# Установка необходимых библиотек
!pip install transformers accelerate bitsandbytes peft datasets torch trl -q

## 2. Импорты и настройка

In [ ]:
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Проверка доступности GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используемое устройство: {device}")

## 3. Загрузка данных
**Подставьте свои данные здесь**

In [ ]:
# === ВАШИ ДАННЫЕ ===
# Вариант 1: Загрузка из CSV/TSV
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

# Вариант 2: Загрузка из Hugging Face
# dataset = load_dataset("dataset_name")

# Вариант 3: Создание своего датасета
data = {
    'text': [
        "Пример текста для обучения 1",
        "Пример текста для обучения 2",
        # Добавьте свои данные
    ]
}
train_df = pd.DataFrame(data)

# Для fine-tuning в формате инструкций
instruction_data = {
    'instruction': ['Вопрос 1', 'Вопрос 2'],
    'response': ['Ответ 1', 'Ответ 2']
}
# train_df = pd.DataFrame(instruction_data)

print(f"Размер датасета: {len(train_df)}")

## 4. Загрузка модели (Inference режим)

In [ ]:
# === НАСТРОЙКИ МОДЕЛИ ===
MODEL_NAME = "microsoft/phi-2"  # Замените на вашу модель
# Примеры: "meta-llama/Llama-2-7b-hf", "mistralai/Mistral-7B-v0.1", "google/gemma-7b"

# Квантизация для экономии памяти (опционально)
USE_QUANTIZATION = True

if USE_QUANTIZATION:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True
    )

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

print("Модель загружена!")

## 5. Генерация текста (Inference)

In [ ]:
# Создание pipeline для генерации
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.95,
    do_sample=True
)

# === ВАШИ ПРОМПТЫ ===
prompts = [
    "Напиши краткое объяснение машинного обучения:",
    "Что такое нейронные сети?"
]

# Генерация
for prompt in prompts:
    outputs = text_generator(prompt)
    print(f"\nПромпт: {prompt}")
    print(f"Ответ: {outputs[0]['generated_text']}")
    print("-" * 80)

## 6. Fine-tuning с LoRA/QLoRA

In [ ]:
# Подготовка модели для обучения с квантизацией
if USE_QUANTIZATION:
    model = prepare_model_for_kbit_training(model)

# Конфигурация LoRA
lora_config = LoraConfig(
    r=16,  # Ранг матриц LoRA
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Модули для применения LoRA
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Подготовка данных для обучения
from datasets import Dataset

# Форматирование данных
def format_instruction(sample):
    return f"""Инструкция: {sample.get('instruction', sample.get('text', ''))}
Ответ: {sample.get('response', '')}"""

# Преобразование в Dataset
train_dataset = Dataset.from_pandas(train_df)

print(f"Размер тренировочного датасета: {len(train_dataset)}")

In [ ]:
# === ПАРАМЕТРЫ ОБУЧЕНИЯ ===
training_args = TrainingArguments(
    output_dir="./llm_finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    logging_steps=10,
    warmup_ratio=0.1,
    optim="paged_adamw_8bit",
)

# Тренер
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    peft_config=lora_config,
    dataset_text_field="text",  # Укажите название колонки с текстом
    tokenizer=tokenizer,
    max_seq_length=512,
)

print("Trainer готов!")

In [ ]:
# Запуск обучения
trainer.train()

# Сохранение модели
trainer.save_model("./llm_finetuned_final")
print("Модель обучена и сохранена!")

## 7. Inference с обученной моделью

In [ ]:
# Загрузка обученной модели
from peft import PeftModel

# Базовая модель
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# Загрузка LoRA адаптера
finetuned_model = PeftModel.from_pretrained(base_model, "./llm_finetuned_final")
finetuned_model = finetuned_model.merge_and_unload()  # Объединение с базовой моделью

# Pipeline для генерации
finetuned_generator = pipeline(
    "text-generation",
    model=finetuned_model,
    tokenizer=tokenizer,
    max_new_tokens=256
)

print("Обученная модель загружена!")

In [ ]:
# === ТЕСТИРОВАНИЕ ===
test_prompts = [
    "Ваш тестовый промпт 1",
    "Ваш тестовый промпт 2"
]

for prompt in test_prompts:
    output = finetuned_generator(prompt)
    print(f"\nПромпт: {prompt}")
    print(f"Ответ: {output[0]['generated_text']}")
    print("-" * 80)

## 8. Создание предсказаний для сабмита

In [ ]:
# === ГЕНЕРАЦИЯ ПРЕДСКАЗАНИЙ ===
# test_df = pd.read_csv('test.csv')  # Загрузите тестовые данные

# predictions = []
# for text in test_df['text_column']:
#     output = finetuned_generator(text, max_new_tokens=100)
#     predictions.append(output[0]['generated_text'])

# # Сохранение
# submission = pd.DataFrame({
#     'id': test_df['id'],
#     'prediction': predictions
# })
# submission.to_csv('llm_submission.csv', index=False)
# print("Предсказания сохранены!")